# Lesson 1: User Authentication

Welcome to the first step in building a full-featured To-Do list application! This unit will focus on **user authentication**, a crucial component for most web applications. We'll walk through how to allow users to register, log in, and log out of the application. Understanding user authentication ensures that your application can manage user-specific data securely and provide each user with a personalized experience.

## What You'll Learn

In this unit, you'll learn how to implement basic user authentication using the **Django framework**. We'll cover the following essential features:

### 1. User Registration

You'll learn how to create a user registration system where new users can sign up with a username, email, and password. Here's a snippet to give you a sneak peek:

```python
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt
from django.contrib.auth.models import User

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        if not username or not email or not password:
            return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)  # Bad request

        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already exists.'}, status=400)

        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()
        return JsonResponse({'message': 'User registered successfully'}, status=201)  # Resource created

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)  # Method not allowed
```

We take the `username`, `email`, and `password` from the request and create a new user in the database. If the username already exists, we return an error message. Otherwise, we create a new user and return a success message.

Notice that we do not have a model for the user. Instead, we use the built-in **User model** from Django's auth module. This model provides all the necessary fields and methods to manage users in the application.

### 2. User Login

We will implement a login mechanism to authenticate users before they access restricted parts of your application.

```python
from django.contrib.auth import authenticate, login

@csrf_exempt
def user_login(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        password = data.get('password')

        if not username or not password:
            return JsonResponse({'error': 'Username and password are required.'}, status=400)

        user = authenticate(request, username=username, password=password)
        if user is not None:
            login(request, user)
            return JsonResponse({'message': 'User logged in successfully'}, status=200)
        return JsonResponse({'error': 'Invalid credentials'}, status=400)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)
```

The code demonstrates how to take the username and password from the request, authenticate the user, and log them in. If the credentials are valid, the user is logged in; otherwise, an error message is returned.

The `authenticate` function checks the provided username and password against the user database. If the credentials are correct, it returns the user object; otherwise, it returns `None`.

The `login` function logs the user in by creating a session for the user. We will discuss **sessions** in more detail in the upcoming units.

### 3. User Logout

You'll also learn how to log users out securely.

```python
from django.contrib.auth import logout

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        logout(request)
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)
```

In this code snippet, we log the user out by calling the `logout` function. This function deletes the user's session, effectively logging them out of the application.

## Why It Matters

User authentication is a fundamental part of web applications for several reasons:

- **Security**: It protects user data and prevents unauthorized access.
- **Personalization**: By authenticating users, you can provide a tailored experience, showing user-specific to-do lists, notifications, and more.
- **Data Management**: Authenticated users allow for better data organization as each user's data is securely separated.

By the end of this unit, you'll have the skills to implement user authentication in your Django applications, making them more robust and secure.

Let's get started and make our To-Do list application more engaging and user-friendly!


## Run Code for User Authentication

Here's a step-by-step overview of how the code works for **User Authentication** in Django, including the **Register**, **Login**, and **Logout** functionalities.

### Full Code Breakdown:

1. **Register View**:
   - This view allows a user to register by submitting a `username`, `email`, and `password`. 
   - If the `username` is already taken or the required fields are missing, it returns an error. Otherwise, a new user is created in the database.
   
   ```python
   @csrf_exempt
   def register(request):
       if request.method == 'POST':
           data = request.POST
           username = data.get('username')
           email = data.get('email')
           password = data.get('password')

           if not username or not email or not password:
               return JsonResponse({'error': 'Username, email, and password are required.'}, status=400)

           if User.objects.filter(username=username).exists():
               return JsonResponse({'error': 'Username already exists.'}, status=400)

           user = User.objects.create_user(username=username, email=email, password=password)
           user.save()
           return JsonResponse({'message': 'User registered successfully'}, status=201)
       
       return JsonResponse({'message': 'Only POST method is allowed'}, status=405)
   ```

2. **Login View**:
   - This view authenticates a user by checking the `username` and `password` provided in the request. 
   - If the credentials are correct, Django creates a session for the user.
   
   ```python
   @csrf_exempt
   def user_login(request):
       if request.method == 'POST':
           data = request.POST
           username = data.get('username')
           password = data.get('password')

           if not username or not password:
               return JsonResponse({'error': 'Username and password are required.'}, status=400)

           user = authenticate(request, username=username, password=password)
           if user is not None:
               login(request, user)
               return JsonResponse({'message': 'User logged in successfully'}, status=200)
           return JsonResponse({'error': 'Invalid credentials'}, status=400)
       
       return JsonResponse({'message': 'Only POST method is allowed'}, status=405)
   ```

3. **Logout View**:
   - This view logs out the user by terminating the session.
   
   ```python
   @csrf_exempt
   def user_logout(request):
       if request.method == 'POST':
           logout(request)
           return JsonResponse({'message': 'User logged out successfully'}, status=200)
       
       return JsonResponse({'message': 'Only POST method is allowed'}, status=405)
   ```

### Testing the Full Code:

We can use Python's `requests` library to simulate HTTP requests for **registration**, **login**, and **logout** as follows:

```python
import requests
import random

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

# Simulating a random user registration and login
rand_number = random.randint(1, 10000) % 1000
username = f"user_{rand_number}"
email = "user@example.com"
password = "passwordthatcosmothinksisokay"

session = requests.Session()

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")

# Logout User
logout_response = logout_user(session)
print(f"Logout Response: {logout_response.status_code} - {logout_response.json()}")
```

### Django URL Configuration:
Ensure that the views are mapped to the correct URLs in `urls.py`:

```python
from django.urls import path
from myapp.views import register, user_login, user_logout

urlpatterns = [
    path('register/', register, name='register'),
    path('login/', user_login, name='login'),
    path('logout/', user_logout, name='logout'),
]
```

### How It Works Together:
- **Register**: First, the code registers a new user with a random username.
- **Login**: After registration, the user logs in with the same credentials.
- **Logout**: Finally, the user logs out, ending their session.

### Conclusion:
By running the provided code, you can see how Django manages the entire authentication process. The registration view creates new users, the login view authenticates them, and the logout view terminates the session. You can easily build upon these features to further enhance your To-Do list application.

## Verify Unique Email on Registration

Here’s how you can complete the `register` function in `views.py` to ensure users can successfully register with a username, email, and password.

### Complete `register` function:

```python
import json
from django.contrib.auth.models import User
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt

@csrf_exempt
def register(request):
    if request.method == 'POST':
        data = request.POST
        username = data.get('username')
        email = data.get('email')
        password = data.get('password')

        # Check if any field is missing
        if not username or not email or not password:
            return JsonResponse({'error': 'All fields (username, email, password) are required.'}, status=400)

        # Check if the username already exists in the database
        if User.objects.filter(username=username).exists():
            return JsonResponse({'error': 'Username already taken.'}, status=400)

        # Check if the email is already registered
        if User.objects.filter(email=email).exists():
            return JsonResponse({'error': 'Email already registered.'}, status=400)

        # Create and save the user
        user = User.objects.create_user(username=username, email=email, password=password)
        user.save()

        return JsonResponse({'message': 'User registered successfully'}, status=201)

    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)
```

### Breakdown:
1. **Field Validation**: Ensures that all fields (username, email, and password) are provided. If any field is missing, a `400` error response is returned with an appropriate message.
2. **Username Uniqueness Check**: It checks if the username already exists in the `User` model. If it does, a `400` error is returned.
3. **Email Uniqueness Check**: Adds another check to prevent duplicate email registrations.
4. **User Creation**: If all checks pass, a new user is created and saved using Django's `create_user` method. A success message with `201` status code is returned.
5. **Only POST Method**: Rejects requests that are not POST with a `405` status.

### Testing the Registration:
The client-side code remains the same. It generates a random username and posts the registration details to the `/register/` endpoint.

```python
import requests
import random

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

rand_number = random.randint(1, 10000) % 1000
username = f"user_{rand_number}"
email = "user@example.com"
password = "passwordthatcosmothinksisokay"
print(username)
session = requests.Session()
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")
```

This setup will ensure users can register with a unique username and email and handle any validation errors appropriately.

## Complete User Registration Function

Here’s how to complete the `user_logout` function in `views.py` to log out the user and return a JSON response indicating successful logout.

### Complete `user_logout` function:

```python
from django.contrib.auth import logout
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        # Log out the user using Django's logout method
        logout(request)
        
        # Return a JSON response indicating successful logout with status code 200
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    # If the request method is not POST, return an error response
    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)
```

### Breakdown:
1. **POST Method Check**: The function first checks if the request method is `POST`. If it's not, it returns a `405` status code indicating that the method is not allowed.
2. **Logging Out**: The `logout(request)` function logs out the user by flushing their session.
3. **Response**: After logging out, the function returns a JSON response with the message `"User logged out successfully"` and a `200` status code.

### Testing the Logout:
You can use the provided client-side script to verify the logout functionality. Here’s how the client-side test looks:

```python
import requests
import random

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

rand_number = random.randint(1, 10000) % 1000
username = f"user_{rand_number}"
email = "user@example.com"
password = "passwordthatcosmothinksisokay"
print(username)
session = requests.Session()

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")

# Logout User
logout_response = logout_user(session)
print(f"Logout Response: {logout_response.status_code} - {logout_response.json()}")
```

### URLs Configuration:

```python
from django.contrib import admin
from django.urls import path
from myapp.views import register, user_login, user_logout

urlpatterns = [
    path('register/', register, name='register'),
    path('login/', user_login, name='login'),
    path('logout/', user_logout, name='logout'),
]
```

This implementation will ensure that users can log out properly, and you will receive a JSON response indicating a successful logout with a `200` status code.

## Implement User Login Function

Here’s how to complete the `user_logout` function in `views.py` to log out the user and return a JSON response indicating successful logout.

### Complete `user_logout` function:

```python
from django.contrib.auth import logout
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        # Log out the user using Django's logout method
        logout(request)
        
        # Return a JSON response indicating successful logout with status code 200
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    # If the request method is not POST, return an error response
    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)
```

### Breakdown:
1. **POST Method Check**: The function first checks if the request method is `POST`. If it's not, it returns a `405` status code indicating that the method is not allowed.
2. **Logging Out**: The `logout(request)` function logs out the user by flushing their session.
3. **Response**: After logging out, the function returns a JSON response with the message `"User logged out successfully"` and a `200` status code.

### Testing the Logout:
You can use the provided client-side script to verify the logout functionality. Here’s how the client-side test looks:

```python
import requests
import random

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

rand_number = random.randint(1, 10000) % 1000
username = f"user_{rand_number}"
email = "user@example.com"
password = "passwordthatcosmothinksisokay"
print(username)
session = requests.Session()

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")

# Logout User
logout_response = logout_user(session)
print(f"Logout Response: {logout_response.status_code} - {logout_response.json()}")
```

### URLs Configuration:

```python
from django.contrib import admin
from django.urls import path
from myapp.views import register, user_login, user_logout

urlpatterns = [
    path('register/', register, name='register'),
    path('login/', user_login, name='login'),
    path('logout/', user_logout, name='logout'),
]
```

This implementation will ensure that users can log out properly, and you will receive a JSON response indicating a successful logout with a `200` status code.

## Complete the user logout function

Here’s how to complete the `user_logout` function in `views.py` to log out the user and return a JSON response indicating successful logout.

### Complete `user_logout` function:

```python
from django.contrib.auth import logout
from django.http import JsonResponse
from django.views.decorators.csrf import csrf_exempt

@csrf_exempt
def user_logout(request):
    if request.method == 'POST':
        # Log out the user using Django's logout method
        logout(request)
        
        # Return a JSON response indicating successful logout with status code 200
        return JsonResponse({'message': 'User logged out successfully'}, status=200)

    # If the request method is not POST, return an error response
    return JsonResponse({'message': 'Only POST method is allowed'}, status=405)
```

### Breakdown:
1. **POST Method Check**: The function first checks if the request method is `POST`. If it's not, it returns a `405` status code indicating that the method is not allowed.
2. **Logging Out**: The `logout(request)` function logs out the user by flushing their session.
3. **Response**: After logging out, the function returns a JSON response with the message `"User logged out successfully"` and a `200` status code.

### Testing the Logout:
You can use the provided client-side script to verify the logout functionality. Here’s how the client-side test looks:

```python
import requests
import random

BASE_URL = "http://127.0.0.1:3000/"

def register_user(session, username, email, password):
    payload = {
        "username": username,
        "email": email,
        "password": password
    }
    response = session.post(BASE_URL + "register/", data=payload)
    return response

def login_user(session, username, password):
    payload = {
        "username": username,
        "password": password
    }
    response = session.post(BASE_URL + "login/", data=payload)
    return response

def logout_user(session):
    response = session.post(BASE_URL + "logout/")
    return response

rand_number = random.randint(1, 10000) % 1000
username = f"user_{rand_number}"
email = "user@example.com"
password = "passwordthatcosmothinksisokay"
print(username)
session = requests.Session()

# Register User
register_response = register_user(session, username, email, password)
print(f"Register Response: {register_response.status_code} - {register_response.json()}")

# Login User
login_response = login_user(session, username, password)
print(f"Login Response: {login_response.status_code} - {login_response.json()}")

# Logout User
logout_response = logout_user(session)
print(f"Logout Response: {logout_response.status_code} - {logout_response.json()}")
```

### URLs Configuration:

```python
from django.contrib import admin
from django.urls import path
from myapp.views import register, user_login, user_logout

urlpatterns = [
    path('register/', register, name='register'),
    path('login/', user_login, name='login'),
    path('logout/', user_logout, name='logout'),
]
```

This implementation will ensure that users can log out properly, and you will receive a JSON response indicating a successful logout with a `200` status code.